說明：把財政部公佈的村里統計數據從html檔案下爬取下來，存成csv。
範圍：88年度-98年度

綜稅所得總額各縣市鄉鎮村里統計分析表
/home/wpc/work/tax_data/html/ias98/ias165A.html
各類所得金額各縣市鄉鎮村里統計表
/home/wpc/wokr/tax_data/html/ias98/ias166A.html

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
county_dict = {'A':'臺北市','B':'臺中市','C':'基隆市','D':'臺南市','E':'高雄市','F':'新北市','G':'宜蘭縣','H':'桃園市','I':'嘉義市','J':'新竹縣','K':'苗栗縣','L':'臺中縣','M':'南投縣','N':'彰化縣','O':'新竹市','P':'雲林縣','Q':'嘉義縣','R':'臺南縣','S':'高雄縣','T':'屏東縣', 'U':'花蓮縣', 'V':'臺東縣','W':'金門縣','X':'澎湖縣','Z':'連江縣'}

In [58]:
url= '/home/wpc/work/tax_data/html/ias88/IAS165.HTM'
f=open(url, 'r',encoding='Big5',errors='ignore')
html=f.read()
soup = BeautifulSoup(html, 'lxml')

In [59]:
df = pd.DataFrame(columns=['縣市','鄉鎮市區', '村里', '納稅單位','綜合所得總額', '平均數','中位數','第一分位數','第三分位數','標準差','變異係數'])
    

for i in range(25):
    num = 1 + i*3
    table = soup.find_all('table')[num]
    county = table.find('tr').text.replace( u'\xa0','').strip().replace(u'\u3000','')
    county = county[4:7]
    num = num +1
    
    table = soup.find_all('table')[num]
    iterrow = iter(table.find_all('tr'))
    next(iterrow)

    for row in iterrow:
        col_num = len(row.find_all(['td','th']))
        if col_num==10:
            new=[county,]
            town=row.find_all(['td','th'])[0].text.strip()
            for col in row.find_all(['td','th']):
                new.append(col.text.strip().replace(' ','').replace(u'\u3000',''))
        elif col_num ==9:
            new=[county,town,]
            for col in row.find_all(['td','th']):
                new.append(col.text.strip().replace(' ','').replace(u'\u3000','')) 
        if col_num ==10 or col_num ==9 :
            df.loc[len(df)] = new

In [61]:
df

,縣市,鄉鎮市區,村里,納稅單位,綜合所得總額,平均數,中位數,第一分位數,第三分位數,標準差,變異係數
0,臺北市,松山區,中崙里,1324,1469272,1110,765,418,1366,1149.98,103.63
1,臺北市,松山區,自強里,2306,2429046,1053,774,432,1328,2578.41,244.78
2,臺北市,松山區,鵬程里,1773,1957995,1104,744,456,1398,1267.57,114.78
3,臺北市,松山區,東榮里,3164,5998595,1896,974,475,1935,8208.71,432.97
4,臺北市,松山區,介壽里,1603,2107954,1315,849,453,1669,1457.42,110.83
...,...,...,...,...,...,...,...,...,...,...,...
8146,連江縣,莒光鄉,其他,2,2774,1387,1387,635,2140,1064.32,76.73
8147,連江縣,東引鄉,中柳村,83,67192,810,714,433,1068,528.20,65.25
8148,連江縣,東引鄉,樂華村,65,56306,866,764,477,1131,442.53,51.09
8149,連江縣,東引鄉,其他,3,3129,1043,1022,285,1822,768.77,73.70


In [60]:
df.loc[(df.村里=='合計') & (df.鄉鎮市區 != '其他'),:]

,縣市,鄉鎮市區,村里,納稅單位,綜合所得總額,平均數,中位數,第一分位數,第三分位數,標準差,變異係數


In [ ]:
df.to_csv('/home/wpc/work/tax_data/income_tax/income_tax_88.csv',index=False)

In [49]:
# 總額分析
# 把資料抓下來
def get_html(letter):
    url = url_core + letter + '.htm'
    html=''
    try:
        f=open(url, 'r',encoding='Big5',errors='ignore')
        html=f.read()
    except:
        pass
    return html

def get_df(html, letter):
    soup = BeautifulSoup(html, 'lxml')
    # 創造空的dataframe
    df = pd.DataFrame(columns=['縣市','鄉鎮市區', '村里', '納稅單位','綜合所得總額', '平均數','中位數','第一分位數','第三分位數','標準差','變異係數'])
    county=county_dict[letter]
    # 該網頁用兩層table包住
    table = soup.find_all('table')[1]
    table = table.find_all('table')[0]
    # 創造空的dataframe
    df = pd.DataFrame(columns=['縣市','鄉鎮市區', '村里', '納稅單位','綜合所得總額', '平均數','中位數','第一分位數','第三分位數','標準差','變異係數'])
    
    # 為了讓迴圈跳過第一row（'村里', '納稅單位','綜合所得總額', '平均數','中位數','第一分位數','第三分位數','標準差','變異係數）
    iterrow = iter(table.find_all('tr'))
    next(iterrow)

    for row in iterrow:
        col_num = len(row.find_all(['td','th']))
        if col_num==10:
            new=[county,]
            town=row.find_all(['td','th'])[0].text.strip().replace(' ','').replace(u'\u3000','')
            for col in row.find_all(['td','th']):
                new.append(col.text.strip().replace(' ','').replace(u'\u3000',''))
        elif col_num ==9:
            new=[county,town,]
            for col in row.find_all(['td','th']):
                new.append(col.text.strip().replace(' ','').replace(u'\u3000','')) 
        if col_num ==10 or col_num ==9 :
            df.loc[len(df)] = new
    return df

In [54]:
url_core =  '/home/wpc/桌面/tax_data/html/ias89/ias165'

# 爬取各縣市資料，在合併成df。其中{'臺南縣':'R','高雄縣':'S','臺中縣':'L'}可能因為縣市合併消失。
dfR= get_df(get_html('R'), 'R')
dfS= get_df(get_html('S'), 'S')
dfL= get_df(get_html('L'), 'L')

dfA= get_df(get_html('A'), 'A')
dfB= get_df(get_html('B'), 'B')
dfC= get_df(get_html('C'), 'C')
dfD= get_df(get_html('D'), 'D')
dfE= get_df(get_html('E'), 'E')
dfF= get_df(get_html('F'), 'F')
dfG= get_df(get_html('G'), 'G')
dfH= get_df(get_html('H'), 'H')
dfI= get_df(get_html('I'), 'I')
dfJ= get_df(get_html('J'), 'J')
dfK= get_df(get_html('K'), 'K')
dfM= get_df(get_html('M'), 'M')
dfN= get_df(get_html('N'), 'N')
dfO= get_df(get_html('O'), 'O')
dfP= get_df(get_html('P'), 'P')
dfQ= get_df(get_html('Q'), 'Q')
dfT= get_df(get_html('T'), 'T')
dfU= get_df(get_html('U'), 'U')
dfV= get_df(get_html('V'), 'V')
dfW= get_df(get_html('W'), 'W')
dfX= get_df(get_html('X'), 'X')
dfZ= get_df(get_html('Z'), 'Z')

df1=pd.concat([dfA,dfB,dfC,dfD,dfE,dfF,dfG,dfH,dfI,dfJ,dfK],ignore_index=True)
df2=pd.concat([dfM,dfN,dfO,dfP,dfQ,dfT,dfU,dfV,dfW,dfX,dfZ],ignore_index=True)
df3=pd.concat([dfR,dfS,dfL],ignore_index=True)
df=pd.concat([df1,df2,df3],ignore_index=True)

In [55]:
dfD

,縣市,鄉鎮市區,村里,納稅單位,綜合所得總額,平均數,中位數,第一分位數,第三分位數,標準差,變異係數
0,臺南市,中區,萬壽里,364,290282,797,508,300,925,898.47,112.66
1,臺南市,中區,文昌里,316,258499,818,603,370,986,754.21,92.20
2,臺南市,中區,中山里,168,139265,829,565,357,916,1133.99,136.80
3,臺南市,中區,中南里,222,203688,918,551,311,1188,1147.21,125.03
4,臺南市,中區,福安里,288,441600,1533,580,355,1085,7301.61,476.19
...,...,...,...,...,...,...,...,...,...,...,...
277,臺南市,安平區,億載里,189,118889,629,538,312,826,448.76,71.34
278,臺南市,安平區,其他,1,198,198,198,198,198,0,0
279,臺南市,安平區,合計,9190,7211128,785,572,344,936,1283.52,163.57
280,臺南市,其他,其他,4,2218,555,459,393,716,261.35,47.13


In [56]:
# 檢查鄉鎮市區數目是否為368。
df.loc[(df.村里=='合計') & (df.鄉鎮市區 != '其他'),:]

,縣市,鄉鎮市區,村里,納稅單位,綜合所得總額,平均數,中位數,第一分位數,第三分位數,標準差,變異係數
34,臺北市,松山區,合計,66270,91472428,1380,810,440,1533,7769.19,562.86
90,臺北市,大安區,合計,106693,165364749,1550,869,457,1708,5803.66,374.45
123,臺北市,中正區,合計,48951,63202122,1291,774,421,1488,7546.66,584.50
161,臺北市,萬華區,合計,53375,42635893,799,577,354,948,1308.25,163.78
189,臺北市,大同區,合計,38019,34891219,918,601,360,1033,2145.87,233.82
...,...,...,...,...,...,...,...,...,...,...,...
8340,臺中縣,龍井鄉,合計,14983,9531861,636,502,307,790,623.79,98.05
8362,臺中縣,霧峰鄉,合計,14853,9849109,663,506,308,832,612.11,92.31
8383,臺中縣,太平市,合計,32809,21338731,650,509,308,805,913.66,140.48
8405,臺中縣,大里市,合計,36755,25263203,687,533,321,852,782.43,113.83


In [62]:
df.to_csv('/home/wpc/work/tax_data/income_tax/income_tax_88.csv',index=False)

In [ ]:
# 收入來源分析
def get_df(html, letter):
    soup = BeautifulSoup(html, 'lxml')
    # 創造空的dataframe
    df = pd.DataFrame(columns=['縣市','鄉鎮市區', '村里', '納稅單位','合計','營利所得','執行業務所得','薪資所得','利息所得','租賃及權利金','財產交易所得','機會中獎所得','股利所得','退職所得','其他所得','稿費所得','申報大於歸戶','薪資淨所得','按薪資淨所得計算之綜合所得','稿費收入'])
    county=county_dict[letter]
    # 該網頁用兩層table包住
    table = soup.find_all('table')[1]
    table = table.find_all('table')[0]
    # 創造空的dataframe

    # 為了讓迴圈跳過第一row（'村里', '納稅單位',..... ）
    iterrow = iter(table.find_all('tr'))
    next(iterrow)
    next(iterrow)
    for row in iterrow:
        col_num = len(row.find_all(['td','th']))
        if col_num==19:
            new=[county,]
            town=row.find_all(['td','th'])[0].text.strip()
            for col in row.find_all(['td','th']):
                new.append(col.text.strip().replace(' ',''))
        elif col_num ==18:
            new=[county,town,]
            for col in row.find_all(['td','th']):
                new.append(col.text.strip().replace(' ','')) 
        if col_num ==19 or col_num ==18 :
            df.loc[len(df)] = new
    return df

In [4]:
url_core =  '/home/wpc/桌面/tax_data/html/ias101/101_165-'

# 爬取各縣市資料，在合併成df。其中{'臺南縣':'R','高雄縣':'S','臺中縣':'L'}可能因為縣市合併消失。
#dfR= get_df(get_html('R'), 'R')
#dfS= get_df(get_html('S'), 'S')
#dfL= get_df(get_html('L'), 'L')

dfA= get_df(get_html('A'), 'A')
dfB= get_df(get_html('B'), 'B')
dfC= get_df(get_html('C'), 'C')
dfD= get_df(get_html('D'), 'D')
dfE= get_df(get_html('E'), 'E')
dfF= get_df(get_html('F'), 'F')
dfG= get_df(get_html('G'), 'G')
dfH= get_df(get_html('H'), 'H')
dfI= get_df(get_html('I'), 'I')
dfJ= get_df(get_html('J'), 'J')
dfK= get_df(get_html('K'), 'K')
dfM= get_df(get_html('M'), 'M')
dfN= get_df(get_html('N'), 'N')
dfO= get_df(get_html('O'), 'O')
dfP= get_df(get_html('P'), 'P')
dfQ= get_df(get_html('Q'), 'Q')
dfT= get_df(get_html('T'), 'T')
dfU= get_df(get_html('U'), 'U')
dfV= get_df(get_html('V'), 'V')
dfW= get_df(get_html('W'), 'W')
dfX= get_df(get_html('X'), 'X')
dfZ= get_df(get_html('Z'), 'Z')

df1=pd.concat([dfA,dfB,dfC,dfD,dfE,dfF,dfG,dfH,dfI,dfJ,dfK],ignore_index=True)
df2=pd.concat([dfM,dfN,dfO,dfP,dfQ,dfT,dfU,dfV,dfW,dfX,dfZ],ignore_index=True)
#df3=pd.concat([dfR,dfS,dfL],ignore_index=True)
df=pd.concat([df1,df2],ignore_index=True)

NameError: name 'get_df' is not defined

In [ ]:
# 檢查鄉鎮市區數目是否為368。
df.loc[(df.村里=='合計') & (df.鄉鎮市區 != '其他'),:]

In [ ]:
df.縣市.value_counts()

In [ ]:
df.to_csv('/home/wpc/桌面/tax_data/temp/income_source_98.csv',index=False)